In [2]:
import xarray as xr
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [1]:
# list the directories of the daily files of ERA5 fluxes and winds 

from glob import glob
from tqdm.notebook import tqdm 

data_directory_lhf = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/latent_heat_flux*.nc')
data_directory_shf = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/sensible_heat_flux*.nc')
data_directory_ssr = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/net_solar_radiation*.nc')
data_directory_str = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/net_thermal_radiation*.nc')
data_directory_ws  = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/winds*.nc')
data_directory_sic = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/sea_ice_cover*.nc')

data_directory = sorted(data_directory_lhf + data_directory_shf + data_directory_ssr + data_directory_str + data_directory_ws + data_directory_sic)

In [3]:
# for each year from 1981 to 2023, calculate the mean summer (DJF) fluxes and winds and put them in a xarray dataset called ds_mean

years = np.arange(1981, 2023, 1)

for y in tqdm(range(years.size)):

    year = years[y]

    flist = [file for file in data_directory if file.endswith(str(year) + '_DJF.nc')]

    ds = xr.open_mfdataset(flist)

    ds = ds.load()

    # remove the data points where there is sea ice

    # Create a mask for sea ice concentration values less than 0.15
    date_mask = ds.siconc < 0.15

    # Create a replacement array with NaN values for the same shape as ds
    replacement_array = xr.full_like(ds.siconc, fill_value=np.nan, dtype=float)

    # Combine the original data and the replacement array
    ds = replacement_array.combine_first(ds.where(date_mask))

    if y == 0:

        ds_mean = ds.mean(dim='time', skipna=True)

    else:

        ds_mean = xr.concat([ds_mean, ds.mean(dim='time', skipna=True)], dim='time')


  0%|          | 0/42 [00:00<?, ?it/s]

In [ ]:
# calculate the net heat flux from the four components

ds_mean['qnet'] = (('time', 'latitude', 'longitude'), ds_mean['sshf'].data + ds_mean['slhf'].data + ds_mean['str'].data + ds_mean['ssr'].data)

In [ ]:
# save the mean fluxes and wind dataset

ds_mean.to_netcdf('../../../data/era5/mean_summer_winds_fluxes_1981_2023.nc')